# Time Series Prediction with BQML and AutoML

**Objectives**
 - Learn how to use BQML to create a classification time-series model using `CREATE MODEL`.
 - Learn how to use BQML to create a linear regression time-series model.
 - Learn how to use AutoML Tables to build a time series model from data in BigQuery.


## Set up environment variables and load necessary libraries

In [2]:
PROJECT = "cloud-training-demos"  # Replace with your PROJECT
REGION = "us-central1"            # Choose an available region for Cloud MLE

In [3]:
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [4]:
!pip freeze | grep google-cloud-bigquery==1.6.1 || pip install google-cloud-bigquery==1.6.1

google-cloud-bigquery==1.6.1


## Review the dataset

In the previous lab we created the data we will use modeling and saved them as tables in BigQuery. Let's examine that table again to see that everything is as we expect. Then, we will build a model using BigQuery ML using this table.

In [10]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  stock_market.percent_change_sp500
LIMIT
  10

,symbol,Date,Open,Close,tomorrow_close,tomo_close_m_close,close_MIN_prior_5_days,close_MIN_prior_20_days,close_MIN_prior_260_days,close_MAX_prior_5_days,...,close_STDDEV_prior_20_days,close_STDDEV_prior_260_days,close_values_prior_260,days_on_market,scaled_change,s_p_scaled_change,normalized_change,company,industry,direction
0,BSX,2006-05-19,20.29,20.15,19.96,-0.19,1.006948,1.006948,1.006948,1.038213,...,0.044825,0.126191,"[29.5, 29.75, 30.42, 30.36, 30.3, 30.75, 30.8,...",3524,-0.009429,-0.003915,-0.005515,Boston Scientific Corp,Health Care,STAY
1,AEP,1996-02-09,43.38,43.63,44.38,0.75,0.991290,0.951180,0.724731,1.008480,...,0.018675,0.073950,"[35.0, 34.88, 35.0, 35.75, 34.88, 35.13, 34.63...",6600,0.017190,0.007740,0.009450,American Electric Power,Utilities,STAY
2,AEP,2003-10-07,29.90,29.88,29.86,-0.02,0.990629,0.958835,0.592035,1.008032,...,0.013977,0.102150,"[27.06, 28.91, 28.61, 28.51, 29.98, 28.74, 27....",8526,-0.000669,-0.005263,0.004594,American Electric Power,Utilities,STAY
3,ABT,2001-12-20,55.25,55.60,56.59,0.99,0.956655,0.953957,0.756475,0.985971,...,0.012940,0.058919,"[54.06, 53.75, 51.19, 51.06, 49.69, 50.06, 50....",4726,0.017806,0.004351,0.013455,Abbott Laboratories,Health Care,UP
4,BAC,1987-10-05,24.37,24.12,24.00,-0.12,1.010365,1.005390,0.875622,1.020730,...,0.019713,0.313248,"[46.5, 46.0, 45.75, 45.13, 45.25, 45.5, 44.38,...",343,-0.004975,-0.027006,0.022030,Bank of America Corp,Financials,UP
5,ADM,1985-06-03,16.87,16.68,16.96,0.28,0.955036,0.893285,0.691247,1.011391,...,0.030916,0.077321,"[12.74, 12.84, 12.56, 12.18, 11.9, 12.0, 11.9,...",547,0.016787,0.003803,0.012983,Archer-Daniels-Midland Co,Consumer Staples,UP
6,BA,1978-07-11,54.38,55.00,54.88,-0.12,0.943273,0.888727,0.429455,0.984182,...,0.023823,0.227507,"[56.63, 57.38, 57.88, 57.88, 58.13, 57.88, 58....",4140,-0.002182,0.003232,-0.005413,Boeing Co,Industrials,STAY
7,ADI,2007-02-15,33.26,33.29,32.98,-0.31,0.988585,0.972364,0.802944,1.004205,...,0.008164,0.095408,"[40.0, 39.77, 40.18, 40.38, 40.9, 40.91, 39.26...",5690,-0.009312,-0.000872,-0.008440,Analog Devices Inc,Information Technology,STAY
8,AA,1978-07-20,42.38,42.63,42.38,-0.25,0.982407,0.958949,0.908984,1.017593,...,0.018717,0.082952,"[52.25, 52.5, 51.13, 51.75, 51.63, 52.75, 53.8...",4147,-0.005864,-0.002856,-0.003008,Alcoa Inc,Materials,STAY
9,AIG,1988-09-26,65.00,64.25,65.12,0.87,0.996109,0.941634,0.785992,1.013541,...,0.018699,0.107580,"[78.75, 79.0, 79.0, 79.25, 81.25, 80.62, 79.37...",1024,0.013541,-0.002306,0.015847,American Intl Group Inc,Financials,UP


## Using BQML

### Create classification model for `direction`

To create a model
1. Use `CREATE MODEL` and provide a destination table for resulting model. Alternatively we can use `CREATE OR REPLACE MODEL` which allows overwriting an existing model.
2. Use `OPTIONS` to specify the model type (linear_reg or logistic_reg). There are many more options [we could specify](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create#model_option_list), such as regularization and learning rate, but we'll accept the defaults.
3. Provide the query which fetches the training data 

Have a look at [Step Two of this tutorial](https://cloud.google.com/bigquery/docs/bigqueryml-natality) to see another example.

**The query will take about two minutes to complete**


We'll start with creating a classification model to predict the `direction` of each stock. 

We'll take a random split using the `symbol` value. With about 500 different values, using `MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1` will give about __ distinct `symbol` values which corresponds to about __ training examples. After taking 70% for training, we will be building a model on about __ training examples.

In [22]:
# TODO 1a
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.direction_model OPTIONS(model_type = "logistic_reg",
    input_label_cols = ["direction"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  tomorrow_close,
  tomo_close_m_close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  direction
FROM
  `stock_market.percent_change_sp500`
WHERE
  tomorrow_close IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*70

""


## Get training statistics and examine training info

After creating our model, we can evaluate the performance using the [`ML.EVALUATE` function](https://cloud.google.com/bigquery-ml/docs/bigqueryml-natality#step_four_evaluate_your_model). With this command, we can find the precision, recall, accuracy F1-score and AUC of our classification model.

In [23]:
# TODO 1b
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.direction_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      tomorrow_close,
      tomo_close_m_close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      direction
    FROM
      `stock_market.percent_change_sp500`
    WHERE
      tomorrow_close IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) > 15*70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*85))

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.495065,0.405386,0.563221,0.395176,1.079252,0.661035


We can also examine the training statistics collected by Big Query. To view training results we use the [`ML.TRAINING_INFO`](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-train) function.

In [24]:
#TODO 1c
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.direction_model`)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,2,0.290389,0.318685,0.8,25136
1,0,1,0.315221,0.328672,0.4,37917
2,0,0,0.343544,0.348294,0.2,33818


### Create regression model for `normalized change`

We can also use BigQuery to train a regression model to predict the normalized change for each stock. To do this in BigQuery we need only change the OPTIONS when calling `CREATE OR REPLACE MODEL`.


In [27]:
#TODO 2a
%%bigquery --project $PROJECT
#standardSQL
CREATE OR REPLACE MODEL
  stock_market.price_model OPTIONS(model_type = "linear_reg",
    input_label_cols = ["normalized_change"]) AS
  -- query to fetch training data
SELECT
  symbol,
  Date,
  Open,
  Close,
  tomorrow_close,
  tomo_close_m_close,
  close_MIN_prior_5_days,
  close_MIN_prior_20_days,
  close_MIN_prior_260_days,
  close_MAX_prior_5_days,
  close_MAX_prior_20_days,
  close_MAX_prior_260_days,
  close_AVG_prior_5_days,
  close_AVG_prior_20_days,
  close_AVG_prior_260_days,
  close_STDDEV_prior_5_days,
  close_STDDEV_prior_20_days,
  close_STDDEV_prior_260_days,
  normalized_change
FROM
  `stock_market.percent_change_sp500`
WHERE
  normalized_change IS NOT NULL
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
  AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*70

""


Just as before we can examine the evaluation metrics for our regression model and examine the training statistics in Big Query

In [43]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.EVALUATE(MODEL `stock_market.price_model`,
    (
    SELECT
      symbol,
      Date,
      Open,
      Close,
      tomorrow_close,
      tomo_close_m_close,
      close_MIN_prior_5_days,
      close_MIN_prior_20_days,
      close_MIN_prior_260_days,
      close_MAX_prior_5_days,
      close_MAX_prior_20_days,
      close_MAX_prior_260_days,
      close_AVG_prior_5_days,
      close_AVG_prior_20_days,
      close_AVG_prior_260_days,
      close_STDDEV_prior_5_days,
      close_STDDEV_prior_20_days,
      close_STDDEV_prior_260_days,
      normalized_change
    FROM
      `stock_market.percent_change_sp500`
    WHERE
      normalized_change IS NOT NULL
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15) = 1
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) > 15*70
      AND MOD(ABS(FARM_FINGERPRINT(symbol)), 15*100) <= 15*85))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,0.010967,0.000291,0.000323,0.007693,0.324587,0.32459


In [44]:
%%bigquery --project $PROJECT
#standardSQL
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `stock_market.price_model`)

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,2,0.000436,0.000518,0.8,10969
1,0,1,0.000477,0.000536,0.4,10201
2,0,0,0.000557,0.000597,0.2,8332


## Train a Time Series model using AutoML Tables

### Step 1. Launch AutoML

#TODO 3

Within the GCP console, navigate to Tables in the console menu.

<img src='assets/console_menu_tables.png' width='50%'>

### Step 2. Create a Dataset

Select New Dataset and give it a name like `stock_market`. In the section on Importing data, select the option to import your data from a BigQuery Table. Fill in the details for your project, the dataset ID, and the table ID.

<img src='assets/import_data_options.png' width='50%'>

### Step 3. Import the Data

Once you have created the dataset you can then import the data. This will take a few minutes.

<img src='assets/importing_data.png' width='50%'>

### Step 4. Train the model.

Once the data has been imported into the dataset. You can examine the Schema of your data, Analyze the properties and values of the features and ultimately Train the model. Here you can also determine the label column and features for training the model. Since we are doing a classifcation model, we'll use `direction` as our target column.

<img src='assets/schema_analyze_train.png' width='80%'>

Under the `Train` tab you can choose the features to use when training. Select the same features as we used above. 

<img src='assets/train_model.png' width='50%'>

### Step 5. Evaluate your model.

Training can take many hours. But once training is complete you can inspect the evaluation metrics of your model. Since this is a classification task, we can also adjust the threshold and explore how different thresholds will affect your evaluation metrics. Also on that page, we can explore the feature importance of the various features used in the model and view confusion matrix for our model predictions.

<img src='assets/eval_metrics.png' width='80%'>

### Step 6. Predict with the trained model. 

Once the model is done training, navigate to the Models page and Deploy the model, so we can test prediction. 

<img src='assets/deploy_model.png' width='80%'>

When calling predictions, you can call batch prediction jobs by specifying a BigQuery table or csv file. Or you can do online prediction for a single instance.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License